In [1]:
from openquake.commonlib import datastore
from nzshm_common.location import location

import h5py
import numpy as np
import pandas as pd
from pathlib import Path

# set all single line variables to be displayed, not just the last line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from openquake.calculators.extract import Extractor
calc_id = '/home/chrisdc/Downloads/calc_1.hdf5'
extractor = Extractor(calc_id)
foo = extractor.get('disagg?kind=Mag&spec=rlzs&imt=PGA&poe_id=0&site_id=0')

In [2]:
### function defined here instead of using
#   from oq_hazard_report.read_oq_hdf5 import *
#   now this requires from nzshm_common.location import location instead...

def find_site_names(sites,dtol=0.001):
    '''
    sets site names as the index for the sites dataframe
    '''

    def name_from_latlon(lat, lon, location_codes, dtol):
        lat_idx = (location_codes['latitude'] >= lat-dtol) & (location_codes['latitude'] <= lat+dtol)
        lon_idx = (location_codes['longitude'] >= lon-dtol) & (location_codes['longitude'] <= lon+dtol)
        idx = lat_idx & lon_idx
        if not True in idx.values:
            return f'Lat: {lat:.2f}, Lon: {lon:.2f}'
        return location_codes[lat_idx & lon_idx].index

    location_codes = {}
    for loc in location.LOCATIONS:
        location_codes[loc['name']] = {'id':loc['id'],'latitude':loc['latitude'],'longitude':loc['longitude']}
    location_codes = pd.DataFrame(location_codes).transpose()


    sites.loc[0,'name'] = 'dummy'
    if 'custom_site_id' in sites:
        for i in sites.index:
            id_idx = location_codes['id'] == sites.loc[i,'custom_site_id']
            if True in id_idx.values:
                try:
                    sites.loc[i,'name'] = location_codes[id_idx].index
                except: #handle duplicate custom_site_ids by looking up by lat lon
                    sites.loc[i,'name'] = name_from_latlon(sites.loc[i,'lat'], sites.loc[i,'lon'], location_codes, dtol)
            else: #if it's not on the list just use the custom_site_id
                sites.loc[i,'name'] = sites.loc[i,'custom_site_id']
    else:
        for i in sites.index:
            sites.loc[i,'name'] = name_from_latlon(sites.loc[i,'lat'], sites.loc[i,'lon'], location_codes, dtol)
    
    return sites.set_index('name')

In [3]:
def extract_disagg(filename):
    
    dstore = datastore.read(filename)
    oqparam = vars(dstore['oqparam'])
    imtls = oqparam['hazard_imtls']
    inv_time = oqparam['investigation_time']
    sites = find_site_names(dstore.read_df('sitecol'),dtol=0.001)
    dstore.close()
    
    if len(sites)==1:
        site = sites.index.values[0]
    else:
        raise NameError('hdf5 includes more than one site location.')
        
    if len(imtls)==1:
        imt = list(imtls.keys())[0]
    else:
        raise NameError('hdf5 includes more than one IMT.')
        
    if len(imtls[imt])==1:
        imtl = imtls[imt][0]
    else:
        raise NameError(f'hdf5 includes more than one IMTL for {imt}.')
    
    with h5py.File(filename) as hf:
        poe = np.squeeze(hf['poe4'][:])
        
        full_disagg = np.squeeze(hf['disagg']['Mag_Dist_TRT_Eps'][:])
        full_disagg_contribution = full_disagg / poe
        
        mag_dist_disagg = np.squeeze(hf['disagg']['Mag_Dist'][:])
        mag_dist_disagg_contribution = mag_dist_disagg / poe

        dist_bin_edges = hf['disagg-bins']['Dist'][:]
        mag_bin_edges = hf['disagg-bins']['Mag'][:]
        eps_bin_edges = hf['disagg-bins']['Eps'][:]

        trt_bins = [x.decode('UTF-8') for x in hf['disagg-bins']['TRT'][:]]
        dist_bins = (dist_bin_edges[1:]-dist_bin_edges[:-1])/2 + dist_bin_edges[:-1]
        eps_bins = (eps_bin_edges[1:]-eps_bin_edges[:-1])/2 + eps_bin_edges[:-1]
        mag_bins = (mag_bin_edges[1:]-mag_bin_edges[:-1])/2 + mag_bin_edges[:-1] 

    disagg = {}
    disagg['site'] = site
    disagg['imt'] = imt
    disagg['imtl'] = imtl
    disagg['poe'] = poe
    disagg['inv_time'] = inv_time
    disagg['disagg_matrix'] = full_disagg
    disagg['bins'] = {'mag_bins':mag_bins,
                      'dist_bins':dist_bins,
                      'trt_bins':trt_bins,
                      'eps_bins':eps_bins}
    disagg['bin_edges'] = {'mag_bin_edges':mag_bin_edges,
                           'dist_bin_edges':dist_bin_edges,
                           'eps_bin_edges':eps_bin_edges}
    
    return disagg

In [4]:
folder = ''
oq_id = 1
filename = f'calc_{oq_id}.hdf5'
filename = str(Path(folder,filename))


In [8]:
### disagg metadata
disagg = extract_disagg(filename)
site = disagg['site']
imt = disagg['imt']
imtl = disagg['imtl']
poe = disagg['poe']
inv_time = disagg['inv_time']
rp = -inv_time/np.log(1-poe)
poe_50 = 100*(1 - np.exp(-50/rp))

In [19]:
shape = disagg['disagg_matrix'].shape
print(f'Shape of disagg_matrix: {shape}')

print()
for bin_type in ['mag','dist','trt','eps']:
    n_bins = len(disagg['bins'][f'{bin_type}_bins'])
    print(f'N {bin_type} bins: {n_bins}')
    print('\tmiddle value of the bins:' + str(disagg['bins'][f'{bin_type}_bins']))


Shape of disagg_matrix: (6, 50, 4, 10)

N mag bins: 6
	middle value of the bins:[7.25 7.75 8.25 8.75 9.25 9.75]
N dist bins: 50
	middle value of the bins:[  5.  15.  25.  35.  45.  55.  65.  75.  85.  95. 105. 115. 125. 135.
 145. 155. 165. 175. 185. 195. 205. 215. 225. 235. 245. 255. 265. 275.
 285. 295. 305. 315. 325. 335. 345. 355. 365. 375. 385. 395. 405. 415.
 425. 435. 445. 455. 465. 475. 485. 495.]
N trt bins: 1
	middle value of the bins:['Subduction Interface']
N eps bins: 4
	middle value of the bins:[-3. -1.  1.  3.]


In [21]:
# check the poe over all bins for the realization

(_,_,_,n_rlz) = disagg['disagg_matrix'].shape
for i_rlz in range(n_rlz):
    poe[i_rlz]

4.866669056036166e-05

5.570595403736611e-06

2.4102213136423245e-07

0.00013301822860911638

1.900886505334931e-05

1.4228828831841511e-06

8.154848569397188e-05

1.6122198819523703e-05

2.2809869899553803e-06

4.157181678876576e-05